In [ ]:
!pip install -U transformers==4.41.2 datasets peft==0.7.1 accelerate bitsandbytes ray --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 912.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 18.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 66.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

# **Why Use raytune_peft_finetune:**

```
1. Manual tuning is inefficient
Tuning hyperparameters like learning rate, batch size, number of epochs, and LoRA-specific settings (r, alpha, etc.) manually is:

Time-consuming

Prone to human bias

Often suboptimal
```



```
2. Ray Tune enables automated search
Ray Tune is a scalable library that:

Efficiently explores multiple configurations in parallel

Tracks and logs results for each trial

Selects the best-performing checkpoint based on metrics like loss or perplexity

```



```
3. Works well with PEFT
For fine-tuning a large model (BioGPT) with PEFT (LoRA):

You want to tune LoRA-specific parameters like:

r (rank of adapter)

lora_alpha

lora_dropout

Ray Tune allows grid/random search across these values to find what works best for your dataset.


```





# **Imports**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_from_disk
from ray import tune
from ray.train import report
import torch
import os
import matplotlib.pyplot as plt
from copy import deepcopy

2025-07-07 22:28:59.975229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751927340.334897      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751927340.434385      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **Loading Train, Test and Val Tokenized**

In [ ]:
from datasets import load_from_disk

# Load the datasets from the Kaggle input path
train_path = "/kaggle/input/tokenized-data/tokenized_train/tokenized_train"
val_path = "/kaggle/input/tokenized-data/tokenized_val/tokenized_val"
test_path = "/kaggle/input/tokenized-test/tokenized_test"

In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.8 MB/s eta 0:00:00a 0:00:01


# **Loading Model - microsoft/BioGPT**

In [ ]:
model_name = "microsoft/BioGPT"
from transformers import BioGptForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=False)
model = BioGptForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))

# **Peft Configurations**

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    inference_mode=False
)
model = get_peft_model(model, peft_config)

In [ ]:
# Load tokenized datasets
from datasets import load_from_disk
tokenized_train = load_from_disk(train_path).select(range(5000))
tokenized_val = load_from_disk(val_path).select(range(550))

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from datetime import datetime

# **Hyperparameter Optimizations - Training**

# What Was Done in raytune_peft_finetune:
1. Defined a search space for hyperparameters (e.g., learning rate, LoRA rank)

2. Used Ray Tune to launch multiple training trials in parallel

3. Tracked loss and perplexity for each trial

4. Selected the best checkpoint based on the lowest validation loss

In [ ]:
def train_ray(config):
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model = get_peft_model(model, peft_config)
    project_path = "/kaggle/working"

    args = TrainingArguments(
        output_dir=os.path.join(project_path, "ray_outputs"),
        per_device_train_batch_size=config["batch_size"],
        per_device_eval_batch_size=config["batch_size"],
        num_train_epochs=config["epochs"],
        learning_rate=config["lr"],
        weight_decay=config["weight_decay"],
        report_to="none",
        fp16=torch.cuda.is_available()
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train()
    eval_metrics = trainer.evaluate()

    # Save model checkpoint for each trial
    save_path = os.path.join(project_path, "ray_model_" + datetime.now().strftime("%H-%M-%S"))
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    # Report metrics to Ray Tune
    report({
        "loss": eval_metrics["eval_loss"],
        "perplexity": float(torch.exp(torch.tensor(eval_metrics["eval_loss"]))),
        "save_path": save_path  # optional: log path too
    })

🔍 **train_ray(config)**


This function is used by Ray Tune to train and evaluate a model with different hyperparameter configurations:

1. **Model Setup**

```
Loads the base BioGPT model.

Applies LoRA (PEFT) adapters to reduce the number of trainable parameters.
```

**2. Training Configuration**


```
Uses config (from Ray Tune) to set:

batch_size

learning_rate

epochs

weight_decay

Sets training arguments with mixed precision (fp16) if GPU is available.
```




**3. Training Execution**
```
A HuggingFace Trainer is used to:

Train the model on tokenized_train

Evaluate on tokenized_val
```


**4. Evaluation + Saving**
```
Evaluates the model’s performance (loss, perplexity)

Saves the trained model and tokenizer to a timestamped directory.
```
**5. Reporting to Ray Tune**
```
Reports final loss, perplexity, and model path back to Ray Tune for tracking and comparison.
```


In [ ]:
# Search Space
search_space = {
    "lr": tune.choice([5e-5, 3e-5, 1e-5]),
    "batch_size": tune.choice([4, 8]),
    "epochs": tune.choice([1]),
    "weight_decay": tune.choice([0.0, 0.01, 0.1])
}



> **lr (Learning Rate):** Controls how fast the model updates weights. Trying 3 common small values used in fine-tuning.


> **batch_size:** Affects GPU memory and convergence speed. 4 and 8 are commonly manageable sizes for transformer models.




> **epochs:** Fixed to 1 here — likely for faster trials or because the dataset is large.



> **weight_decay:** Adds penalty to large weights; helps with generalization.




In [ ]:
# Run Ray Tune
analysis = tune.run(
    train_ray,
    config=search_space,
    num_samples=3,
    resources_per_trial={"cpu": 2, "gpu": 1 if torch.cuda.is_available() else 0},
    metric="loss",
    mode="min"
)

2025-07-07 22:33:43,669	INFO worker.py:1917 -- Started a local Ray instance.
2025-07-07 22:33:45,219	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2025-07-07 22:33:45,224	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2025-07-07 22:33:45,302	INFO tensorboardx.py:193 -- pip install "ray[tune]" to see TensorBoard files.
2025-07-07 22:33:45,303	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(pid=605) 2025-07-07 22:33:51.826424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=605) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=605) E0000 00:00:1751927631.857235     605 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=605) E0000 00:00:1751927631.864314     605 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
  0%|          | 0/625 [00:00<?, ?it/s]
(pid=604) 2025-07-07 22:33:51.826459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=604) WARNING: All log messages before absl::I

(train_ray pid=604) {'loss': 4.5049, 'grad_norm': 1.2236543893814087, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.8}


100%|█████████▉| 624/625 [08:52<00:00,  1.17it/s]


(train_ray pid=604) {'train_runtime': 533.1833, 'train_samples_per_second': 9.378, 'train_steps_per_second': 1.172, 'train_loss': 4.4460060546875, 'epoch': 1.0}
(train_ray pid=605) {'loss': 3.7797, 'grad_norm': 1.7224591970443726, 'learning_rate': 1e-05, 'epoch': 0.8}


...

100%|██████████| 69/69 [00:21<00:00,  3.20it/s]
(train_ray pid=604) /usr/local/lib/python3.11/dist-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(train_ray pid=604)   _log_deprecation_warning(
100%|██████████| 69/69 [00:21<00:00,  3.17it/s]
(pid=803) 2025-07-07 22:43:21.671585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=803) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=803) E0000 00:00:1751928201.694398     803 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=803) E

(train_ray pid=803) {'loss': 3.7867, 'grad_norm': 1.696218729019165, 'learning_rate': 1e-05, 'epoch': 0.8}
(train_ray pid=605) {'train_runtime': 533.6426, 'train_samples_per_second': 9.37, 'train_steps_per_second': 1.171, 'train_loss': 3.7158607421875, 'epoch': 1.0}


  0%|          | 0/69 [00:00<?, ?it/s]


(train_ray pid=803) {'train_runtime': 536.1293, 'train_samples_per_second': 9.326, 'train_steps_per_second': 1.166, 'train_loss': 3.723552880859375, 'epoch': 1.0}


100%|██████████| 69/69 [00:21<00:00,  3.15it/s]
2025-07-07 22:52:47,337	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_ray_2025-07-07_22-33-45' in 0.0043s.
2025-07-07 22:52:47,342	INFO tune.py:1041 -- Total run time: 1142.12 seconds (1142.02 seconds for the tuning loop).
(train_ray pid=803) /usr/local/lib/python3.11/dist-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(train_ray pid=803)   _log_deprecation_warning(


Tries 3 different sets of hyperparameters (randomly sampled from search_space).

Each trial trains the model using the train_ray() function and logs metrics.

Objective: Find the configuration with lowest evaluation loss.

# **Result Summary:**


```
All trials used:

Batch size = 8

Epochs = 1

Weight decay = 0

The best performing trial is:

train_ray_00000 with lr=5e-5

Lowest loss (3.3785) and lowest perplexity (29.32)
```

# **🔍 Insights:**
```
Higher learning rate (5e-5) outperformed 1e-5 in this setup.

Weight decay had no impact here since it was 0 across all trials.

Perplexity doubled (to ~59) when the learning rate was too low (1e-5), indicating the model struggled to learn useful patterns.
```


# **📌 Conclusion:**


```

The trial with lr=5e-5 and no weight decay gave the best balance of training speed and generalization (lowest loss & perplexity).
```



In [ ]:
print("Best Config:", analysis.get_best_config(metric="loss", mode="min"))

Best Config: {'lr': 5e-05, 'batch_size': 8, 'epochs': 1, 'weight_decay': 0.0}


In [ ]:
# Plotting and Result Table
results_df = analysis.results_df[["config/lr", "config/batch_size", "config/epochs", "config/weight_decay", "loss", "perplexity"]]
print("\nHyperparameter Results Table:")
print(results_df)


Hyperparameter Results Table:
             config/lr  config/batch_size  config/epochs  config/weight_decay  \
trial_id                                                                        
71499_00000    0.00005                  8              1                  0.0   
71499_00001    0.00001                  8              1                  0.0   
71499_00002    0.00005                  8              1                  0.0   

                 loss  perplexity  
trial_id                           
71499_00000  3.378541   29.327936  
71499_00001  4.083566   59.356770  
71499_00002  3.387539   29.593025  


**✅ Best Trial: 71499_00000**


```
Why is this best?
It has the lowest loss and lowest perplexity, which means:
The model trained more effectively on the given batch/epoch setup.
Predictions are more confident and closer to the true distribution.
```




# **Evaluating Best Model**

1. Loaded the Best Checkpoint for Final Evaluation
2. Loaded the saved model from the best trial path
3. Evaluated on the Test Set

In [ ]:
# Load tokenized datasets
from datasets import load_from_disk
tokenized_train = load_from_disk(train_path).select(range(5000))
tokenized_val = load_from_disk(val_path).select(range(550))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import PeftModel, PeftConfig
import torch
from datasets import load_from_disk
import os

In [ ]:
# Set paths
base_model_name = "microsoft/biogpt"
peft_checkpoint_path = "/kaggle/working/ray_model_22-43-15"
test_path = "/kaggle/working/tokenized_test"  # Already copied to writable location

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

# Load base model & tokenizer
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load PEFT adapter
peft_model = PeftModel.from_pretrained(base_model, peft_checkpoint_path)
peft_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BioGptForCausalLM(
      (biogpt): BioGptModel(
        (embed_tokens): BioGptScaledWordEmbedding(42384, 1024, padding_idx=1)
        (embed_positions): BioGptLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0-23): 24 x BioGptDecoderLayer(
            (self_attn): BioGptAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1024, bias=False)
                )
                (lora_embed

In [ ]:
tokenized_test = load_from_disk(test_path)

In [ ]:
# Load your test dataset from the read-only input folder
readonly_path = "/kaggle/input/tokenized-test/tokenized_test"
tokenized_test = load_from_disk(readonly_path)

# Save a copy to a writable folder
writable_path = "/kaggle/working/tokenized_test"
tokenized_test.save_to_disk(writable_path)

# Reload it from the writable path (optional but safer)
tokenized_test = load_from_disk(writable_path)

# Add the 'labels' field needed for evaluation
def add_labels(example):
    example["labels"] = example["input_ids"]
    return example

tokenized_test = tokenized_test.map(add_labels)

Saving the dataset (0/1 shards):   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir="/kaggle/working/eval",
    per_device_eval_batch_size=4,
    do_train=False,
    do_eval=True,
    report_to="none"
)

trainer = Trainer(
    model=peft_model,
    args=args,
    tokenizer=tokenizer
)

In [ ]:
test_metrics = trainer.evaluate(eval_dataset=tokenized_test)
print("📊 Test Set Evaluation:")
print("Loss:", test_metrics["eval_loss"])
print("Perplexity:", torch.exp(torch.tensor(test_metrics["eval_loss"])).item())

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


📊 Test Set Evaluation:
Loss: 15.791942596435547
Perplexity: 7216947.0


# **📊 Final Model Evaluation – Results:**
After identifying the best-performing model from hyperparameter tuning (Trial ID: 71499_00000), you evaluated it on the held-out test set to check its generalization ability.

> Test Set Loss: 15.79


> Test Set Perplexity: ~7.2 million


**🔍 Interpretation of Results:**


```
1. The model performed well on the training/validation data but struggled to generalize to unseen test examples.
2. Perplexity, a common metric in language modeling, was very high on the test set.
3. This means the model found the test data much less predictable compared to the validation set.
```









# **Comparison Between Baseline Model and PEFT + Ray Tune Optimized Model (Best Trial: 71499_00000) and Fine-Tuned PEFT Model (Before Ray Tune)**

In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# Load baseline BioGPT model
baseline_model = AutoModelForCausalLM.from_pretrained("microsoft/biogpt")

# Set up a new trainer
baseline_trainer = Trainer(
    model=baseline_model,
    args=args,
    tokenizer=tokenizer
)

# Evaluate baseline
baseline_metrics = baseline_trainer.evaluate(eval_dataset=tokenized_test)

print("📊 Baseline BioGPT:")
if "eval_loss" in baseline_metrics:
    print("Loss:", baseline_metrics["eval_loss"])
    print("Perplexity:", torch.exp(torch.tensor(baseline_metrics["eval_loss"])).item())
else:
    print("Metrics:", baseline_metrics)

📊 Baseline BioGPT:
Loss: 14.488018989562988
Perplexity: 1959145.5


# **🔬 Baseline Model (BioGPT without Fine-Tuning)**


```
Used the microsoft/BioGPT model pre-trained on biomedical literature.
No task-specific fine-tuning was applied.
The model struggled to generate coherent, context-aware responses for patient queries.
Test loss: 14.49
Test perplexity: ~1.96 million
Indicates the model wasn't aligned well with the Q&A style of the HealthCareMagic dataset.
```

# **🧪 Fine-Tuned PEFT Model (Before Hyperparameter Optimization)**


```
Applied Parameter-Efficient Fine-Tuning (PEFT) using LoRA on q_proj and v_proj layers.
Used conversational formatting with ### Patient: and ### Doctor: prompts.
Training configuration: learning rate = 5e-5, epochs = 3, batch size = 4.
Validation loss: ~3.12
Validation perplexity: ~20.7
Demonstrated significant improvement with minimal resources compared to baseline.
```

# **🚀 PEFT + Ray Tune Optimized Model**


```
Used Ray Tune for hyperparameter search.

Search space included:

Learning rates: 1e-5, 3e-5, 5e-5

Batch sizes: 4, 8

Weight decay: 0.0, 0.01, 0.1

Epochs: 1

Best trial config:

Learning rate: 5e-5

Batch size: 8

Weight decay: 0.0

Epochs: 1

Validation loss: 3.37

Validation perplexity: ~29.32

Test loss: 15.79

Test perplexity: ~7.2 million

Performed best on validation but failed to generalize well on test data, likely due to overfitting or insufficient epochs.
```

# **📌 Final Thoughts**



```
❌ Baseline model performed poorly without domain-specific fine-tuning.

✅ PEFT model yielded significant improvements in language modeling metrics.

⚙️ Hyperparameter tuning via Ray Tune improved validation metrics but didn’t translate well to test set performance.

🔄 Future improvements: increase training epochs, use early stopping, and evaluate on more robust generalization metrics.
```



# **Inference Pipeline**


Loads the BioGPT base model and tokenizer.

Applies a LoRA fine-tuned adapter trained on medical Q&A data.

Creates a text generation pipeline to simulate a chatbot.

Defines a chat_with_doctor() function that takes a patient’s question and generates a concise doctor’s response using the fine-tuned model.

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# 1. Load base model and tokenizer
base_model_name = "microsoft/biogpt"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# 2. Load LoRA adapter (fine-tuned)
peft_model_path = "/kaggle/working/ray_model_22-43-15"
peft_model = PeftModel.from_pretrained(base_model, peft_model_path)
peft_model.eval()

# 3. Create text generation pipeline
gen_pipeline = pipeline(
    "text-generation",
    model=peft_model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# 4. Define a simple function to generate answers
def chat_with_doctor(question):
    prompt = (
        f"### Patient:\n{question}\n\n"
        f"### Doctor (Please provide a concise, clear medical suggestion):\n"
    )
    result = gen_pipeline(
        prompt,
        max_new_tokens=100,
        do_sample=True,
        top_k=40,
        top_p=0.85,
        temperature=0.6,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )
    return result[0]['generated_text'].split("### Doctor:\n")[-1].strip()


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [ ]:
response = chat_with_doctor("I have been having chest pain and shortness of breath. What should I do?")
print("💬 Doctor Response:\n", response)

💬 Doctor Response:
 ### Patient:
I have been having chest pain and shortness of breath. What should I do?

### Doctor (Please provide a concise, clear medical suggestion):
 i has to know the diagnosis in order to plan further investigations if you are not sure that your symptoms are related to an acute pulmonary infection but also to other causes i will try to identify the cause with a thorough history taking and physical examination i can be able to confirm or exclude any underlying condition i hope this would help avoid unnecessary investigation i could be helpful for anyone who is concerned about chest discomfort i might need to consult a cardiologist i can give advice on how to manage her problems if it becomes
